In [1]:
! ls

clean_tweet_subc_withuser_f.csv  RNN_suba_may_god_bless_me.ipynb
idsMatrix.npy			 test_clean_c.csv
model_final_subc.ipynb		 test_clean_withuser.csv
output_rnn_subc.csv		 Untitled.ipynb
output_rnn_subc.ipynb		 wordList-200.npy
pix				 wordVectors-200.npy
RNN_100_subc.ipynb


In [2]:
import numpy as np
import pandas as pd

csv = 'clean_tweet_subc_withuser_f.csv'
df = pd.read_csv(csv)
df.head()

,id,text,subc
0,46191,user user how does that make him the best in t...,2
1,81896,user know whole lotta people spitting whole lo...,2
2,25291,user see it damn shame how delusional he is he...,2
3,38487,user is it gun control or anti haven seen sing...,2
4,27483,user user user could be either liberals like t...,2


In [3]:
len(df.subc)

3876

In [4]:
df.subc.value_counts()

0    2407
2    1074
1     395
Name: subc, dtype: int64

In [5]:
df.subc[3875]

0

In [6]:
1073 + 395

1468

In [7]:
'''0-1073 --- 2
    1074 - 1468 --- 1
    1469 - 3875 --- 0
    '''

'0-1073 --- 2\n    1074 - 1468 --- 1\n    1469 - 3875 --- 0\n    '

In [8]:
wordsList = np.load('wordList-200.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors-200.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


In [9]:
import tensorflow as tf
maxSeqLength = 65 #Maximum length of sentence
numDimensions = 200 #Dimensions for each word vector
firstSentence = np.zeros((maxSeqLength), dtype='int32')
firstSentence[0] = wordsList.index("i")
firstSentence[1] = wordsList.index("thought")
firstSentence[2] = wordsList.index("the")
firstSentence[3] = wordsList.index("movie")
firstSentence[4] = wordsList.index("was")
firstSentence[5] = wordsList.index("incredible")
firstSentence[6] = wordsList.index("and")
firstSentence[7] = wordsList.index("inspiring")
#firstSentence[8] and firstSentence[9] are going to be 0
print(firstSentence.shape)
print(firstSentence) #Shows the row index for each word

(65,)
[   41   804     0  1005    15  7446     5 13767     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]


In [10]:
#from os import listdir
#from os.path import isfile, join
#positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
#negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
k = df.text
#k = np.array(k)

for i in range(len(k)) :
  line = str(k[i])
  counter = len(line.split())
  numWords.append(counter)
  
  

#for pf in positiveFiles:
#    with open(pf, "r", encoding='utf-8') as f:
#        line=f.readline()
#        counter = len(line.split())
#        numWords.append(counter)       
#print('Positive files finished')

#for nf in negativeFiles:
#    with open(nf, "r", encoding='utf-8') as f:
#        line=f.readline()
#        counter = len(line.split())
#        numWords.append(counter)  
#print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))



The total number of files is 3876
The total number of words in the files is 89070
The average number of words in the files is 22.979876160990713


In [11]:
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)

(65, 200)


In [12]:
k = df.text

In [13]:
maxSeqLength = 65

In [14]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

In [15]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0

for i in range(len(k)) :
  indexCounter = 0
  line = str(k[i])
  cleanedLine = cleanSentences(line)
  split = cleanedLine.split()
  for word in split:
    try :
      ids[fileCounter][indexCounter] = wordsList.index(word)
    except ValueError:
      ids[fileCounter][indexCounter] = 399999
      
    indexCounter = indexCounter + 1  
    if indexCounter >= maxSeqLength:
      break
  fileCounter = fileCounter + 1   
      
      
      

# for pf in positiveFiles:
#    with open(pf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 

# for nf in negativeFiles:
#    with open(nf, "r") as f:
#        indexCounter = 0
#        line=f.readline()
#        cleanedLine = cleanSentences(line)
#        split = cleanedLine.split()
#        for word in split:
#            try:
#                ids[fileCounter][indexCounter] = wordsList.index(word)
#            except ValueError:
#                ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
#            indexCounter = indexCounter + 1
#            if indexCounter >= maxSeqLength:
#                break
#        fileCounter = fileCounter + 1 
# #Pass into embedding function and see if it evaluates. 

# np.save('idsMatrix', ids)

In [18]:
len(ids)

3876

In [19]:
np.save('idsMatrix', ids)

In [20]:
df.head()

,id,text,subc
0,46191,user user how does that make him the best in t...,2
1,81896,user know whole lotta people spitting whole lo...,2
2,25291,user see it damn shame how delusional he is he...,2
3,38487,user is it gun control or anti haven seen sing...,2
4,27483,user user user could be either liberals like t...,2


In [21]:
labels1 = []
p = df.subc
for i in range(len(p)):
    o = int(p[i])
    if o == 0 :
        labels1.append([1,0,0])
    else :
        if o == 1 :
            labels1.append([0,1,0])
        else :
            labels1.append([0,0,1])
                    
                

In [22]:
labels2 = []
p = df.subc
for i in range(len(p)):
    o = int(p[i])
    if o == 0 :
        labels2.append([1,0,0])
    else :
        if o == 1 :
            labels2.append([0,1,0])
        else :
            labels2.append([0,0,1])
                    
                

In [23]:
labels1[1469]

[1, 0, 0]

In [24]:
len(labels2)

3876

In [25]:
df['rnn'] = labels1

In [26]:
df.tail()

,id,text,subc,rnn
3871,69534,user hahaha the goofball is losing money and i...,0,"[1, 0, 0]"
3872,38374,they were liberals when you were still suckin ...,0,"[1, 0, 0]"
3873,89954,user user user it started with cameron and may...,0,"[1, 0, 0]"
3874,12391,user user you are just another liberal pawn us...,0,"[1, 0, 0]"
3875,31616,bitch just be minding her business,0,"[1, 0, 0]"


In [27]:
df.head()

,id,text,subc,rnn
0,46191,user user how does that make him the best in t...,2,"[0, 0, 1]"
1,81896,user know whole lotta people spitting whole lo...,2,"[0, 0, 1]"
2,25291,user see it damn shame how delusional he is he...,2,"[0, 0, 1]"
3,38487,user is it gun control or anti haven seen sing...,2,"[0, 0, 1]"
4,27483,user user user could be either liberals like t...,2,"[0, 0, 1]"


In [28]:
df.text[1350]

'user user yeah keep mocking her right into yes vote'

In [29]:
ids[1350]

array([ 4832,  4832,  8738,   578, 23764,    71,   248,    75,  2772,
         538,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0], dtype=int32)

In [30]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 3 == 0): 
            num = randint(0,1013)
            labels.append(labels1[num])
        else:
            if (i % 3 == 1): 
                num = randint(1074,1439)
                labels.append(labels1[num])
            else:
                if (i % 3 == 2): 
                    num = randint(1469,3816)
                    labels.append(labels1[num])
        arr[i] = ids[num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 3 == 0): 
            num = randint(1013,1073)
            labels.append(labels2[num])
        else:
            if (i % 3 == 1): 
                num = randint(1439,1469)
                labels.append(labels2[num])
            else:
                if (i % 3 == 2): 
                    num = randint(3816,3876)
                    labels.append(labels2[num])
        arr[i] = ids[num]
    return arr, labels




In [31]:
batchSize = 24
a,b = getTrainBatch()

In [32]:
len(a)

24

In [33]:
batchSize = 24
lstmUnits = 64
numClasses = 3
iterations = 100000

In [34]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

In [35]:
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

In [36]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [37]:
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

In [38]:
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [39]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [40]:
import datetime

sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [41]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
  nextBatch, nextBatchLabels = getTrainBatch();
  sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
  if (i % 50 == 0):
    summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
    writer.add_summary(summary, i)
  if (i % 10000 == 0 and i != 0):
    save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
    print("saved to %s" % save_path)
writer.close()

/home/preeti/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


saved to models/pretrained_lstm.ckpt-10000
saved to models/pretrained_lstm.ckpt-20000
saved to models/pretrained_lstm.ckpt-30000
saved to models/pretrained_lstm.ckpt-40000


KeyboardInterrupt: 

In [58]:
len(ids)

3876

In [59]:
'''validation'''

'validation'

In [60]:
import tensorflow as tf

In [61]:
numDimensions = 200
maxSeqLength = 65
batchSize = 24
lstmUnits = 64
numClasses = 3
iterations = 100000

In [62]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

In [63]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-40000


/home/preeti/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [64]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch();
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

Accuracy for this batch: 62.5
Accuracy for this batch: 66.66666865348816
Accuracy for this batch: 50.0
Accuracy for this batch: 50.0
Accuracy for this batch: 45.83333432674408
Accuracy for this batch: 50.0
Accuracy for this batch: 37.5
Accuracy for this batch: 62.5
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 54.16666865348816
